In [1]:
import pickle
import pandas as pd
import os           
import numpy as np
import re
import itertools
import plotly.express as px
#import brightway2 as bw 
import random 
import string

In [2]:
import sys
sys.path.append('../../Module')  #two levels up & then down to Module folder
#from recipe_mod import *
from recipe_mod_resIC import *

In [3]:
lcia_name = "Mineral resource scarcity"
SP_name = "Mineral resource scarcity"
rivm_name = "Mineral resource scarcity"
olca_filepathname = "../../Data_rawCFs/olca_recipe_MinRec_saved.dat"
bw2_filepathname = "../../Data_rawCFs/BW2_recipe_Mineral Resource Scarcity.dat"

# used in final calculation to create dict names
pairwise_name = ["SPvsRIVM", "SPvsBW2", "SPvsopenLCA", "openLCAvsBW2", "openLCAvsRIVM", "BW2vsRIVM"]

### read in SP LCIA

In [4]:
SP_recipe = pd.read_excel("../../Data_rawCFs/SP_ReCiPe.xlsx", sheet_name=SP_name, 
                                  header=[0], converters={'CAS':str})

print(len(SP_recipe))
SP_recipe.fillna(''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(9)), inplace=True)
#to assign a random string_num to CAS with NA, so that they're not the "SAME" CAS when compared to other sources:
#print ([x for x in SP_recipe["CAS"] if '-' not in x ] [0]) 
SP_recipe.columns

143


Index(['Compartment', 'Subcompartment', 'Substance', 'CAS', 'Factor', 'Unit'], dtype='object')

In [5]:
SP_recipe = SP_recipe.rename(columns={"Compartment": "category", "Subcompartment": "subcategory", 
                               "Substance": "flow", "Factor":"value", "CAS": "cas_number" })
SP_recipe = SP_recipe.rename(str.lower, axis='columns')

In [6]:
SP_recipe = SP_recipe.rename(columns={"Compartment": "category", "Subcompartment": "subcategory", 
                               "Substance": "flow", "Factor":"value", "CAS": "cas_number" })
SP_recipe = SP_recipe.rename(str.lower, axis='columns')
# lower case for all EFs
SP_recipe['flow'] = SP_recipe['flow'].str.lower()
SP_recipe.head()

,category,subcategory,flow,cas_number,value,unit
0,Raw,(unspecified),aluminium,007429-90-5,0.16900,kg Cu eq / kg
1,Raw,(unspecified),antimony,007440-36-0,0.57200,kg Cu eq / kg
2,Raw,(unspecified),arsenic,007440-38-2,0.13100,kg Cu eq / kg
3,Raw,(unspecified),barite,013462-86-7,0.02280,kg Cu eq / kg
4,Raw,(unspecified),bauxite,001318-16-7,0.00458,kg Cu eq / kg


### read in openlca LCIA

In [7]:
olca_recipe = unpickle_data(olca_filepathname)
olca_recipe.fillna(''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(9)), inplace=True)
#print ([x for x in olca_recipe["CAS_NUMBER"] if '-' not in x ] [0]) 
print(len(olca_recipe))
olca_recipe = olca_recipe.rename(str.lower, axis='columns')

316


In [8]:
#olca_recipe['flow'] = olca_recipe['flow'].str.split(',').str[0]
olca_recipe['flow'] = olca_recipe["flow"].str.lower()
olca_recipe.head()

,impact_method,impact_category_uuid,impact_category,reference_unit,flow_uuid,flow,cas_number,subcategory,category,unit,value
0,ReCiPe 2016 Midpoint (H),f9e7dd9d-f3a3-318e-b458-6452fbdcbd8f,Mineral resource scarcity,kg Cu eq,4718c804-ed44-464c-acbe-c5cc51b7580b,"tio2, 54% in ilmenite, 18% in crude ore",013463-67-7,land,Resource,kg,0.515
1,ReCiPe 2016 Midpoint (H),f9e7dd9d-f3a3-318e-b458-6452fbdcbd8f,Mineral resource scarcity,kg Cu eq,ba2da2fe-3420-45d1-9d1b-58b9e99714eb,"rhodium, rh 1.6e-7%, in mixed ore",QVIXQS2UD,in ground,Resource,kg,"6,340"
2,ReCiPe 2016 Midpoint (H),f9e7dd9d-f3a3-318e-b458-6452fbdcbd8f,Mineral resource scarcity,kg Cu eq,f7d19093-ce81-36a2-ac82-586aebc1f374,rutile,QVIXQS2UD,in ground,Resource,kg,0.179
3,ReCiPe 2016 Midpoint (H),f9e7dd9d-f3a3-318e-b458-6452fbdcbd8f,Mineral resource scarcity,kg Cu eq,537b8312-3f27-4d3c-859f-55d3f944f42d,"limestone, in ground",001317-65-3,in ground,Resource,kg,0.0202
4,ReCiPe 2016 Midpoint (H),f9e7dd9d-f3a3-318e-b458-6452fbdcbd8f,Mineral resource scarcity,kg Cu eq,7ae398b5-8532-11e0-9d78-0800200c9a66,iron ore,QVIXQS2UD,in ground,Resource,kg,0.0175


### read in BW2 LCIA

In [9]:
bw_recipe = unpickle_data(bw2_filepathname)
bw_recipe.fillna(''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(9)), inplace=True)
#print ([x for x in olca_recipe["CAS_NUMBER"] if '-' not in x ] [0]) 
print(len(bw_recipe))
bw_recipe.columns  #already named in the right format and lower.str : flow, cas_number

161


Index(['cas_number', 'flow', 'flow_uuid', 'type', 'category', 'subcategory',
       'unit', 'value'],
      dtype='object')

In [10]:
#bw_recipe['flow'] = bw_recipe['flow'].str.split(',').str[0]
bw_recipe['flow'] = bw_recipe['flow'].str.lower()
bw_recipe

,cas_number,flow,flow_uuid,type,category,subcategory,unit,value
0,007440-36-0,"antimony, in ground",47262180-8308-5d4c-9332-c77617e032ef,natural resource,natural resource,in ground,kilogram,0.571853
1,007440-38-2,"arsenic, in ground",e16fd15c-0ebc-55ba-8d3b-9704f13663cb,natural resource,natural resource,in ground,kilogram,0.130982
2,001302-78-9,"clay, bentonite, in ground",93806a54-46f5-409c-99c5-4144a1e73b5d,natural resource,natural resource,in ground,kilogram,0.010806
3,007440-41-7,"beryllium, in ground",68f7bc03-0665-55a1-bd99-96530eab30e2,natural resource,natural resource,in ground,kilogram,76.651856
4,007440-69-9,"bismuth, in ground",0124b342-4bdd-5cbf-ba2a-dce8a259755c,natural resource,natural resource,in ground,kilogram,3.202969
...,...,...,...,...,...,...,...,...
156,007440-27-9,"terbium, in ground",6ec6a8a8-4c94-5bc5-93c5-62928fcf3935,natural resource,natural resource,in ground,kilogram,3.186281
157,007440-30-4,"thulium, in ground",a34e010c-6f68-5c79-8b45-2955b91f7dc2,natural resource,natural resource,in ground,kilogram,3.186281
158,007440-64-4,"ytterbium, in ground",ea659cca-fe3d-5b03-90d2-60719a862874,natural resource,natural resource,in ground,kilogram,3.186281
159,007440-67-7,"zirconium, 50% in zircon, 0.39% in crude ore, ...",fcee6eab-e906-4ddf-bc14-2b131b937893,natural resource,natural resource,in ground,kilogram,0.174993


### read in raw RIVM_ReCiPe,  delete "*" for some flows suffix

In [11]:
rivm_recipe = pd.read_excel("../../Data_rawCFs/ReCiPe2016_CFs_v1.1_20180117.xlsx", sheet_name=rivm_name, 
                                  header=[2], converters={'CAS':str})
rivm_recipe = rivm_recipe.rename(columns={"Mineral resource": "flow", "Hierarchist":"value"})
rivm_recipe = rivm_recipe.rename(str.lower, axis='columns')

In [12]:
rivm_recipe['flow'] = rivm_recipe['flow'].str.split('*').str[0]
rivm_recipe['flow'] = rivm_recipe['flow'].str.lower()
rivm_recipe

,flow,chemical element,individualist,value,egalitarian
0,aluminium,Al,0.100740,0.169339,0.169339
1,antimony,Sb,1.031823,0.571853,0.571853
2,arsenic,As,0.088871,0.130982,0.130982
3,ball clay,NaN,0.003859,0.007092,0.007092
4,barite,NaN,0.013563,0.022817,0.022817
...,...,...,...,...,...
70,garnets,NaN,0.029944,0.047644,0.047644
71,gemstones,NaN,12491.707091,7994.824659,7994.824659
72,platinum-group metals,NaN,5552.480459,3762.311267,3762.311267
73,rare earth metals,NaN,2.752674,3.186281,3.186281


### 3. final calculation, pair-wise comparison, corr. matrix

In [13]:
# create 6 empty dict: dict_SPvsRIVM; dict_SPvsBW2; dict_SPvsopenLCA; dict_openLCAvsBW2; dict_openLCAvsRIVM; dict_BW2vsRIVM
# each dict with five keys and null values (to be assigned later)
for name in pairwise_name:
    #print(name)
    keys = [name+"_commonEF_sumtable", name+"_commonEF_list", name + "_result_diffEFs", name + "_result_corr" ]
    globals()['dict_%s' % name] = dict.fromkeys(keys, None)

In [15]:
k_list = list(dict_SPvsRIVM)
dict_SPvsRIVM[k_list[0]], dict_SPvsRIVM[k_list[1]] = res_compare_included_EF(SP_recipe, "flow", rivm_recipe,  "flow", to_print = "no" )
if len(dict_SPvsRIVM[k_list[1]] ) == 0: 
    dict_SPvsRIVM[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'RIVM_value': np.NaN }, index=[0])
else:
    dict_SPvsRIVM[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_recipe, "flow", "value", rivm_recipe, "flow", "value", dict_SPvsRIVM[k_list[1]])[0] , 
         'RIVM_value': final_EF_combined(SP_recipe, "flow", "value", rivm_recipe, "flow", "value", dict_SPvsRIVM[k_list[1]])[1] }, 
        index = dict_SPvsRIVM[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 62


In [16]:
k_list = list(dict_SPvsBW2)
dict_SPvsBW2[k_list[0]],dict_SPvsBW2[k_list[1]] = res_compare_included_EF(SP_recipe, "flow", bw_recipe,  "flow", to_print = "no" )
if len(dict_SPvsBW2[k_list[1]] ) == 0: 
    dict_SPvsBW2[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'BW2_value': np.NaN }, index=[0])
else:
    dict_SPvsBW2[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_recipe, "flow", "value", bw_recipe, "flow", "value", dict_SPvsBW2[k_list[1]])[0] , 
         'BW2_value': final_EF_combined(SP_recipe, "flow", "value", bw_recipe, "flow", "value", dict_SPvsBW2[k_list[1]])[1] }, 
        index = dict_SPvsBW2[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 0


In [17]:
k_list = list(dict_SPvsopenLCA)
dict_SPvsopenLCA[k_list[0]],dict_SPvsopenLCA[k_list[1]] = res_compare_included_EF(SP_recipe, "flow", olca_recipe,  "flow", to_print = "no" )
if len(dict_SPvsopenLCA[k_list[1]] ) == 0: 
    dict_SPvsopenLCA[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'olca_value': np.NaN }, index=[0])
else:
    dict_SPvsopenLCA[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_recipe, "flow", "value", olca_recipe, "flow", "value", dict_SPvsopenLCA[k_list[1]])[0] , 
         'olca_value': final_EF_combined(SP_recipe, "flow", "value", olca_recipe, "flow", "value", dict_SPvsopenLCA[k_list[1]])[1] },
        index = dict_SPvsopenLCA[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 143


In [18]:
k_list = list(dict_openLCAvsBW2)
dict_openLCAvsBW2[k_list[0]],dict_openLCAvsBW2[k_list[1]] = res_compare_included_EF(olca_recipe, "flow_uuid", bw_recipe, "flow_uuid", to_print = "no" )

if len(dict_openLCAvsBW2[k_list[1]] ) == 0: 
    dict_openLCAvsBW2[k_list[2]] = pd.DataFrame ({'olca_value': np.NaN , 'BW2_value': np.NaN }, index=[0])
else:
    dict_openLCAvsBW2[k_list[2]] = pd.DataFrame (
        {'olca_value': final_EF_combined(olca_recipe, "flow_uuid", "value", bw_recipe, "flow_uuid", "value", 
                                       dict_openLCAvsBW2[k_list[1]])[0] , 
         'BW2_value': final_EF_combined(olca_recipe, "flow_uuid", "value", bw_recipe, "flow_uuid", "value",
                                         dict_openLCAvsBW2[k_list[1]])[1] },
        index =  dict_openLCAvsBW2[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 124
Different value for a same flow: f77aacc3-2c22-4bda-99ab-fe1110a1b891
Different value for a same flow: 99c56f25-9ebb-4e6a-a3e2-e4dc61e9d697
Different value for a same flow: f77aacc3-2c22-4bda-99ab-fe1110a1b891
Different value for a same flow: 99c56f25-9ebb-4e6a-a3e2-e4dc61e9d697


In [19]:
k_list = list(dict_openLCAvsRIVM)
dict_openLCAvsRIVM[k_list[0]],dict_openLCAvsRIVM[k_list[1]] = res_compare_included_EF(olca_recipe, "flow", rivm_recipe, "flow", to_print = "no" )

if len(dict_openLCAvsRIVM[k_list[1]] ) == 0: 
    dict_openLCAvsRIVM[k_list[2]] = pd.DataFrame ({'olca_value': np.NaN , 'RIVM_value': np.NaN }, index=[0])
else:
    dict_openLCAvsRIVM[k_list[2]] = pd.DataFrame (
        {'olca_value': final_EF_combined(olca_recipe, "flow", "value", rivm_recipe, "flow", "value", 
                                       dict_openLCAvsRIVM[k_list[1]])[0] , 
         'BW2_value': final_EF_combined(olca_recipe, "flow", "value", rivm_recipe, "flow", "value",
                                         dict_openLCAvsRIVM[k_list[1]])[1] },
        index = dict_openLCAvsRIVM[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 62


In [20]:
k_list = list(dict_BW2vsRIVM)
dict_BW2vsRIVM[k_list[0]],dict_BW2vsRIVM[k_list[1]] = res_compare_included_EF(bw_recipe, "flow", rivm_recipe, "flow", to_print = "no" )
if len(dict_BW2vsRIVM[k_list[1]] ) == 0: 
    dict_BW2vsRIVM[k_list[2]] = pd.DataFrame ({'BW2_value': np.NaN , 'RIVM_value': np.NaN }, index=[0])
else:
    dict_BW2vsRIVM[k_list[2]] = pd.DataFrame (
        {'BW2_value': final_EF_combined(bw_recipe, "flow", "value", rivm_recipe, "flow", "value", 
                                       dict_BW2vsRIVM[k_list[1]])[0] , 
         'RIVM_value': final_EF_combined(bw_recipe, "flow", "value", rivm_recipe, "flow", "value",
                                         dict_BW2vsRIVM[k_list[1]])[1] },
        index = dict_BW2vsRIVM[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 0


In [22]:
for name in pairwise_name:
    new_df = globals()['dict_%s' % name][name + "_result_diffEFs"]
    try:
        new_df_corr = new_df.corr()
    except: 
        error
    print(new_df_corr)
    globals()['dict_%s' % name][name + "_result_corr"] = new_df_corr

            SP_value  RIVM_value
SP_value         1.0         1.0
RIVM_value       1.0         1.0
           SP_value  BW2_value
SP_value        NaN        NaN
BW2_value       NaN        NaN
            SP_value  olca_value
SP_value         1.0         1.0
olca_value       1.0         1.0
            olca_value  BW2_value
olca_value         1.0        1.0
BW2_value          1.0        1.0
            olca_value  BW2_value
olca_value         1.0        1.0
BW2_value          1.0        1.0
            BW2_value  RIVM_value
BW2_value         NaN         NaN
RIVM_value        NaN         NaN


In [23]:
# saving each dict_ to a single worksheet, different rows , # saving to sub-folder "results/Resources_IC/"

if not os.path.exists("results/Resources_IC"):
    os.makedirs("results/Resources_IC")

filename = lcia_name + "_pairwise_comp_result.xlsx"
writer = pd.ExcelWriter("results/Resources_IC/" + filename, engine='xlsxwriter')

for name in pairwise_name:
    sum_table = globals()['dict_%s' % name][name + "_commonEF_sumtable"]
    new_df = globals()['dict_%s' % name][name + "_result_diffEFs"]
    new_df_corr = globals()['dict_%s' % name][name + "_result_corr"]

    sum_table.to_excel(writer,sheet_name=name,startrow=0 , startcol=0)   # write first  (3rows)
    new_df_corr.to_excel(writer,sheet_name=name,startrow=5 , startcol=0) # write next   (3rows)
    new_df.to_excel(writer,sheet_name=name,startrow=10 , startcol=0)     # finally, write the resulting EFs 
     
writer.save()